# Boltzmann Machines

### Importing the libraries

In [3]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable

### Importing the dataset

In [5]:
movies = pd.read_csv('ml-1m/movies.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')
users = pd.read_csv('ml-1m/users.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')
ratings = pd.read_csv('ml-1m/ratings.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')

### Preparing the training set and the test set

In [7]:
training_set = pd.read_csv('ml-100k/u1.base', delimiter = '\t')
training_set = np.array(training_set, dtype = 'int')
test_set = pd.read_csv('ml-100k/u1.test', delimiter = '\t')
test_set = np.array(test_set, dtype = 'int')

### Getting the number of users and movies

In [9]:
nb_users = int(max(max(training_set[:,0]), max(test_set[:,0])))
nb_movies = int(max(max(training_set[:,1]), max(test_set[:,1])))

### Converting the data into an array with users in rows and movies in columns

In [11]:
def convert(data):
    new_data = []
    for id_users in range(1, nb_users + 1):
        id_movies = data[:,1][data[:,0] == id_users]
        id_ratings = data[:,2][data[:,0] == id_users]
        ratings = np.zeros(nb_movies)
        ratings[id_movies - 1] = id_ratings
        new_data.append(list(ratings))
    return new_data
training_set = convert(training_set)
test_set = convert(test_set)

### Converting the data into Torch tensors

In [13]:
training_set = torch.FloatTensor(training_set)
test_set = torch.FloatTensor(test_set)

### Converting the ratings into binary ratings 1 (Liked) or 0 (Not Liked)

In [15]:
training_set[training_set == 0] = -1
training_set[training_set == 1] = 0
training_set[training_set >= 2] = 1
#training_set[training_set >= 3] = 1
test_set[test_set == 0] = -1
test_set[test_set == 1] = 0
test_set[test_set >= 2] = 1
#test_set[test_set >= 3] = 1

### Creating the architecture of the Neural Network

In [17]:
class RBM():
    
    def __init__(self, nv, nh):
        self.W = torch.randn(nh, nv)
        self.a = torch.randn(1, nh)
        self.b = torch.randn(1, nv)
    
    def sample_h(self, x):
        wx = torch.mm(x, self.W.t())
        activation = wx + self.a.expand_as(wx)
        p_h_given_v = torch.sigmoid(activation)
        return p_h_given_v, torch.bernoulli(p_h_given_v)
    
    def sample_v(self, y):
        wy = torch.mm(y, self.W)
        activation = wy + self.b.expand_as(wy)
        p_v_given_h = torch.sigmoid(activation)
        return p_v_given_h, torch.bernoulli(p_v_given_h)

    # Approximate the gradient for hidden and visible nodes using Contrastive Divergence algorithm
    def train(self, v0, vk, ph0, phk):
        self.W += torch.mm(v0.t(), ph0).t() - torch.mm(vk.t(), phk).t()
        self.b += torch.sum((v0 - vk), 0)
        self.a += torch.sum((ph0 - phk), 0)

nv = len(training_set[0])
nh = 100
batch_size = 100
rbm = RBM(nv, nh)

### Training the RBM

In [19]:
nb_epoch = 10
for epoch in range(1, nb_epoch + 1):
    train_loss = 0
    s = 0
    for id_user in range(0, nb_users - batch_size, batch_size): # Batch size used only in training set
        vk = training_set[id_user:id_user+batch_size]
        v0 = training_set[id_user:id_user+batch_size]
        ph0,_ = rbm.sample_h(v0)
        for k in range(10): # Gibbs sampling using Markov Chain Monte Carlo (MCMC) theory
            _,hk = rbm.sample_h(vk)
            _,vk = rbm.sample_v(hk)
            vk[v0<0] = v0[v0<0]
        phk,_ = rbm.sample_h(vk)
        rbm.train(v0, vk, ph0, phk)
        train_loss += torch.mean(torch.abs(v0[v0>=0] - vk[v0>=0]))
        s += 1
    print('epoch: '+str(epoch)+' loss: '+str(train_loss/s))

epoch: 1 loss: tensor(0.2414)
epoch: 2 loss: tensor(0.0924)
epoch: 3 loss: tensor(0.1074)
epoch: 4 loss: tensor(0.0930)
epoch: 5 loss: tensor(0.0973)
epoch: 6 loss: tensor(0.0976)
epoch: 7 loss: tensor(0.0979)
epoch: 8 loss: tensor(0.1008)
epoch: 9 loss: tensor(0.0980)
epoch: 10 loss: tensor(0.0982)


**Result Interpretation**

Training set loss of 0.0982 means the model performs with almost 91% accuracy in predicting whether the user liked the movie or not.

### Testing the RBM

In [22]:
test_loss = 0
s = 0
for id_user in range(nb_users):
    v = training_set[id_user:id_user+1]
    vt = test_set[id_user:id_user+1] # Predicting ratings in the test set using the input in previous row training set
    if len(vt[vt>=0]) > 0: # Taking only ratings that are existent
        _,h = rbm.sample_h(v) # Sample hidden nodes
        _,v = rbm.sample_v(h) # Sample visible nodes using above hidden nodes
        test_loss += torch.mean(torch.abs(vt[vt>=0] - v[vt>=0])) # vt is the prediction/target and v is the actual
        s += 1
print('test loss: '+str(test_loss/s))

test loss: tensor(0.0967)


**Result Interpretation**

Looking at the test set loss of 0.0967, we can say that the model predicts movie ratings correctly 91% of the time.